In [36]:
import cPickle as pickle
from pprint import pprint
with open('thesis/results/features_2016-12-14_10-15-43.pkl') as f:
    data, configuration = pickle.load(f)

print "PROJECTS:"
for p_name in data.iterkeys():
    print p_name

print
print "CONFIG:"
pprint(configuration)

PROJECTS:
Sowbug3
Cam1
Camera3
Zebrafish

CONFIG:
{'rf_class_weight': 'balanced_subsample',
 'rf_criterion': 'entropy',
 'seed': 42,
 'test_split_method': 'random'}


In [37]:
p_names = [p for p in data.iterkeys()]
print p_names
splits = [s for s in data[p_names[0]].iterkeys()]
print splits

# for p_name, data in data.iteritems():
#     print p_name

['Sowbug3', 'Cam1', 'Camera3', 'Zebrafish']
[0.95, 'layer', 0.9, 0.8, 0.99]
